In [1]:
import os
from dotenv import load_dotenv
import mysql.connector
import requests
import pandas as pd
from datetime import datetime, timedelta
from IPython.display import HTML, display
from openpyxl import Workbook
from openpyxl.drawing.image import Image
from mysql.connector import Error

In [2]:
load_dotenv()

True

In [3]:
# MySQL credentials from environment variables
MYSQL_HOST = os.getenv('MYSQL_HOST')
MYSQL_USER = os.getenv('MYSQL_USER')
MYSQL_PASSWORD = os.getenv('MYSQL_PASSWORD')
MYSQL_DATABASE = os.getenv('MYSQL_DATABASE')

In [8]:
from datetime import datetime, timedelta

# Get today's date
today = datetime.today()

# Get the date 7 days from today
next_7_days = today + timedelta(days=6)

# Format the dates in the correct format (e.g., 'YYYY-MM-DD')
start_date = today.strftime('%Y-%m-%dT00:00:00Z')
end_date = next_7_days.strftime('%Y-%m-%dT23:59:59Z')

print("Start Date:", start_date)
print("End Date:", end_date)

Start Date: 2025-02-13T00:00:00Z
End Date: 2025-02-19T23:59:59Z


In [9]:
# Weather API
# Load the API key from environment variables
TOMORROW_API_KEY = os.getenv('TOMORROW_API_KEY')

# API pull with parameters
params = {
    "apikey": TOMORROW_API_KEY,       
    "city": "San Diego", 
    "startDateTime": start_date,  # Start date (today)
    "endDateTime": end_date      # End date (7 days later)                
}

# Define the API endpoint
TOMORROW_API_Endpoint = 'https://api.tomorrow.io/v4/weather/forecast?location=San%20Diego&fields=temperature_2m,weather_code&apikey=8zUbskgrkdcP29AtpCqw8XBCutnp5fJh'

# Send the request to the API
response = requests.get(TOMORROW_API_Endpoint, params=params)

In [10]:
# Check if the request was successful
if response.status_code == 200:
    print("API call succeeded!")
    data = response.json()  # The API response in JSON format
    print(data)             # Print the data (you'll see the events info here)
else:
    print(f"API call failed with status code: {response.status_code}")
    print("Response details:", response.text)  # Print more details on the failure

API call succeeded!
{'timelines': {'minutely': [{'time': '2025-02-14T05:37:00Z', 'values': {'cloudBase': 0.2, 'cloudCeiling': None, 'cloudCover': 45, 'dewPoint': 19.1, 'freezingRainIntensity': 0, 'hailProbability': 10.8, 'hailSize': 2.01, 'humidity': 91, 'precipitationProbability': 0, 'pressureSeaLevel': 1020, 'pressureSurfaceLevel': 895, 'rainIntensity': 0, 'sleetIntensity': 0, 'snowIntensity': 0, 'temperature': 20.6, 'temperatureApparent': 20.6, 'uvHealthConcern': 0, 'uvIndex': 0, 'visibility': 16, 'weatherCode': 1101, 'windDirection': 6, 'windGust': 4.3, 'windSpeed': 1.2}}, {'time': '2025-02-14T05:38:00Z', 'values': {'cloudBase': 0.2, 'cloudCeiling': None, 'cloudCover': 46, 'dewPoint': 19.1, 'freezingRainIntensity': 0, 'hailProbability': 87.5, 'hailSize': 9.55, 'humidity': 91, 'precipitationProbability': 0, 'pressureSeaLevel': 1020, 'pressureSurfaceLevel': 895, 'rainIntensity': 0, 'sleetIntensity': 0, 'snowIntensity': 0, 'temperature': 20.5, 'temperatureApparent': 20.5, 'uvHealthCon

In [11]:
# Extract temperature, humidity, rain, and weather code for each hour
weather_data = []

# Weather code to integer mapping (direct integers from API)
weather_code_icons = {
    1100: "mostly_clear_day.svg",  # Clear or mostly clear
    1101: "partly_cloudy.svg",     # Partly cloudy
    1102: "mostly_cloudy.svg",     # Mostly cloudy
    1001: "cloudy.svg",            # Cloudy
    2100: "fog_light.svg",         # Light fog
    2000: "fog.svg",               # Fog
    4000: "drizzle.svg",           # Drizzle
    4200: "rain_light.svg",        # Light rain
    4001: "rain.svg",              # Rain
    4201: "rain_heavy.svg",        # Heavy rain
    6000: "freezing_drizzle.svg",  # Freezing drizzle
    6200: "freezing_rain_light.svg",  # Light freezing rain
    6001: "freezing_rain.svg",     # Freezing rain
    6201: "freezing_rain_heavy.svg",  # Heavy freezing rain
    8000: "tstorm.svg",            # Thunderstorm
}

# Base URL for weather icons
base_url = "https://raw.githubusercontent.com/tomorrow-io-api/tomorrow-weather-codes/main/icons/"

# Parse the data
for interval in data['timelines']['hourly']:
    time = interval['time']
    temperature = interval['values']['temperature']
    temperatureApparent = interval['values']['temperatureApparent']
    humidity = interval['values']['humidity']
    rain = interval['values']['rainIntensity']
    windSpeed = interval['values']['windSpeed']
    weather_codeday = interval['values']['weatherCode']  # Integer weather code

    # Get the icon filename based on the integer weather code
    icon_filename = weather_code_icons.get(weather_codeday, "default.png")

    # Construct the full icon URL
    weather_icon = f"{base_url}{icon_filename}"

    weather_data.append({
        'time': time,
        'temperature': temperature,
        'temperatureApparent': temperatureApparent,
        'humidity': humidity,
        'rain': rain,
        'windSpeed': windSpeed,
        'weather_code': weather_codeday,
        'weather_icon': weather_icon
    })

# Convert values to a DataFrame for easier visualization
weather_df = pd.DataFrame(weather_data)
print(weather_df)

# Function to display icons in the DataFrame
def path_to_image_html(path):
    return f'<img src="{path}" width="50" height="50">'

# Apply the function to the 'weather_icon' column
weather_df['weather_icon'] = weather_df['weather_icon'].apply(path_to_image_html)

# Display the DataFrame with icons
display(HTML(weather_df.to_html(escape=False)))

                     time  temperature  temperatureApparent  humidity  rain  \
0    2025-02-14T05:00:00Z         20.8                 20.8        91  0.00   
1    2025-02-14T06:00:00Z         18.5                 18.5        96  0.00   
2    2025-02-14T07:00:00Z         18.2                 18.2        96  0.00   
3    2025-02-14T08:00:00Z         17.5                 17.5        97  0.00   
4    2025-02-14T09:00:00Z         17.1                 17.1        93  0.00   
..                    ...          ...                  ...       ...   ...   
115  2025-02-19T00:00:00Z         22.1                 22.1        69  0.00   
116  2025-02-19T01:00:00Z         21.2                 21.2        75  0.00   
117  2025-02-19T02:00:00Z         20.4                 20.4        81  0.02   
118  2025-02-19T03:00:00Z         19.5                 19.5        87  0.02   
119  2025-02-19T04:00:00Z         19.2                 19.2        90  0.02   

     windSpeed  weather_code  \
0          1.6     

,time,temperature,temperatureApparent,humidity,rain,windSpeed,weather_code,weather_icon
0,2025-02-14T05:00:00Z,20.8,20.8,91,0.00,1.6,1100,
1,2025-02-14T06:00:00Z,18.5,18.5,96,0.00,0.6,1001,
2,2025-02-14T07:00:00Z,18.2,18.2,96,0.00,1.0,1001,
3,2025-02-14T08:00:00Z,17.5,17.5,97,0.00,1.0,1100,
4,2025-02-14T09:00:00Z,17.1,17.1,93,0.00,0.8,1100,
5,2025-02-14T10:00:00Z,16.9,16.9,89,0.00,0.7,1001,
6,2025-02-14T11:00:00Z,16.6,16.6,84,0.00,0.8,1101,
7,2025-02-14T12:00:00Z,16.7,16.7,81,0.00,0.6,1100,
8,2025-02-14T13:00:00Z,17.4,17.4,77,0.00,1.2,1100,
9,2025-02-14T14:00:00Z,19.5,19.5,70,0.00,1.8,1100,


In [14]:
# Function to connect to MySQL database
def connect_to_mysql():
    try:
        connection = mysql.connector.connect(
            host=MYSQL_HOST,  # Use variables loaded from dotenv
            database=MYSQL_DATABASE,
            user=MYSQL_USER,
            password=MYSQL_PASSWORD,
        )
        if connection.is_connected():
            print("Connected to MySQL database")
            return connection
    except Error as e:
        print(f"Error: {e}")
        return None
    
# Convert 'time' from 'YYYY-MM-DDTHH:MM:SSZ' to 'YYYY-MM-DD HH:MM:SS'
weather_df['time'] = pd.to_datetime(weather_df['time']).dt.strftime('%Y-%m-%d %H:%M:%S')

In [15]:
# Function to insert data into MySQL
def insert_data_to_mysql(df, connection):
    try:
        if connection.is_connected():
            cursor = connection.cursor()
            insert_query = """
                INSERT INTO weather_forecasts (
                    forecast_time, forecast_temperature, forecast_temp_apparent, forecast_humidity, rain_intensity, forecast_winds, weather_code, weather_icon
                )
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
            """
            # Insert each row from the DataFrame
            for row in df.itertuples(index=False):
                cursor.execute(insert_query, (
                    row.time, 
                    row.temperature, 
                    row.temperatureApparent, 
                    row.humidity, 
                    row.rain, 
                    row.windSpeed, 
                    row.weather_code, 
                    row.weather_icon
                ))
            connection.commit()
            print(f"Inserted {cursor.rowcount} rows successfully")
    except Error as e:
        print(f"Error during data insertion: {e}")
    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()
            print("MySQL connection is closed")

# Ensure connection is established
mysql_connection = connect_to_mysql()




if mysql_connection:
    insert_data_to_mysql(weather_df, mysql_connection)
else:
    print("Failed to connect to MySQL database.")

Connected to MySQL database
Inserted 1 rows successfully
MySQL connection is closed
